In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("/content/manufacturing_data.csv")
df.head(2)

,Record Date,Fiscal Quarter,Production Dept,Day Of Week,Team,Planned Efficiency,Standard Minute Value,Work In Progress,Over Time Minutes,Performance Bonus,Idle Minutes,Idle Workers,Style Change Count,Worker Count,Efficiency Score
0,2/24/2015,Q4,Stitching Unit,Tuesday,Line-S1,0.6,22.53,708.0,5040,0,0.0,0,Minor Style Revision,42.0,0.268214
1,1/19/2015,Q3,Stitching Unit,Monday,Line-F3,0.8,11.41,1028.0,4380,50,0.0,0,No Style Change,31.0,0.800359


Changing the DateTime in a correct Format.

In [ ]:
df['Record Date'] = pd.to_datetime(df['Record Date'], format='%m/%d/%Y')


In [ ]:
df.head(3)

,Record Date,Fiscal Quarter,Production Dept,Day Of Week,Team,Planned Efficiency,Standard Minute Value,Work In Progress,Over Time Minutes,Performance Bonus,Idle Minutes,Idle Workers,Style Change Count,Worker Count,Efficiency Score
0,2015-02-24,Q4,Stitching Unit,Tuesday,Line-S1,0.6,22.53,708.0,5040,0,0.0,0,Minor Style Revision,42.0,0.268214
1,2015-01-19,Q3,Stitching Unit,Monday,Line-F3,0.8,11.41,1028.0,4380,50,0.0,0,No Style Change,31.0,0.800359
2,2015-01-06,Q1,Finishing & Quality,Tuesday,Line-C2,0.8,2.90,NaN,1440,0,0.0,0,No Style Change,8.0,0.681061


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler

**Looking** for the **Nan Values.**

In [ ]:
missing_val = df.isnull().sum()
print(missing_val)

Record Date                0
Fiscal Quarter             0
Production Dept            0
Day Of Week                0
Team                       0
Planned Efficiency         0
Standard Minute Value      0
Work In Progress         506
Over Time Minutes          0
Performance Bonus          0
Idle Minutes               0
Idle Workers               0
Style Change Count         0
Worker Count               0
Efficiency Score           0
dtype: int64


**Checking** the **count of Unique values** in **Production department.**

In [ ]:
df['Production Dept'].unique()

array(['Stitching Unit', ' Finishing & Quality ', 'Finishing & Quality'],
      dtype=object)

**Removing** the **First and Last space** with **Strip Function.**

In [ ]:
df['Production Dept'] = df['Production Dept'].str.strip()
count_with_space = df[df['Production Dept'] == ' Finishing & Quality '].shape[0]
df['Production Dept'].value_counts()
print("Rows with leading space:", count_with_space)

Rows with leading space: 0


**Copying** the **Main Data** into **df_stitch**

In [ ]:
df_stitch = df.copy()

**Dropping the Values** which has **Nan Values** from **Work in Progress column.**

In [ ]:
df_stitch = df_stitch.dropna(subset=['Work In Progress'])
df_stitch.head(2)

,Record Date,Fiscal Quarter,Production Dept,Day Of Week,Team,Planned Efficiency,Standard Minute Value,Work In Progress,Over Time Minutes,Performance Bonus,Idle Minutes,Idle Workers,Style Change Count,Worker Count,Efficiency Score
0,2015-02-24,Q4,Stitching Unit,Tuesday,Line-S1,0.6,22.53,708.0,5040,0,0.0,0,Minor Style Revision,42.0,0.268214
1,2015-01-19,Q3,Stitching Unit,Monday,Line-F3,0.8,11.41,1028.0,4380,50,0.0,0,No Style Change,31.0,0.800359


**Counting** of Number of Rows of Stitct.

In [ ]:
df_stitch.shape

(691, 15)

**Label Encoding** for **certain Columns.**

In [ ]:
cols_encode = ['Fiscal Quarter', 'Production Dept', 'Day Of Week', 'Team', 'Style Change Count']
le = LabelEncoder()

for col in cols_encode:
  df_stitch[col] = le.fit_transform(df_stitch[col])

**Dropping** the **Record Date Column** as this will not affect our **Training and Testing**.

In [ ]:
df_stitch.drop(columns=['Record Date'], inplace = True)

**Assigning** **X** , **Y** and **Target Values.**

In [ ]:
target = 'Efficiency Score'
x = df_stitch.drop(columns=[target])
y = df_stitch[target]

**Train Test Split**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

**Scaling** with **Standard Scaling** and **Robust Scaling.**

In [ ]:
standard_cols = ['Planned Efficiency', 'Standard Minute Value', 'Over Time Minutes', 'Idle Minutes', 'Worker Count']

robust_cols = ['Work In Progress','Performance Bonus']

scaler_standard = StandardScaler()
scaler_robust = RobustScaler()

#fits only on training data.
#Learns the pattern/Statistics from your training data.
scaler_standard.fit(x_train[standard_cols])
scaler_robust.fit(x_train[robust_cols])

#transforms on both train and testing Data.
#Applies that learned transformation.
x_train[standard_cols] = scaler_standard.transform(x_train[standard_cols])
x_test[standard_cols] = scaler_standard.transform(x_test[standard_cols])

x_train[robust_cols] = scaler_robust.transform(x_train[robust_cols])
x_test[robust_cols] = scaler_robust.transform(x_test[robust_cols])

In [ ]:
x_train.head(2)

,Fiscal Quarter,Production Dept,Day Of Week,Team,Planned Efficiency,Standard Minute Value,Work In Progress,Over Time Minutes,Performance Bonus,Idle Minutes,Idle Workers,Style Change Count,Worker Count
167,3,0,2,6,-0.210612,-1.102085,0.532847,-0.161355,-0.151515,-0.072729,0,2,-1.860146
127,1,0,3,7,0.725442,0.403967,0.366006,1.419419,1.303030,-0.072729,0,2,0.700454


Performing **Linear Regression**

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred_lr = lr.predict(x_test)

mse_lr = mean_squared_error(y_test,y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression Results")
print("Mean Squared Error: ", mse_lr)
print("R2 score: ", r2_lr)

Linear Regression Results
Mean Squared Error:  0.005132908176519028
R2 score:  0.7100987108760786


Performing **Decision Regression**

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(x_train, y_train)
y_pred_dt = dt.predict(x_test)

mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print("Decision Tree Results")
print("Mean Squared Error: ", mse_dt)
print("R2 Score: ", r2_dt)

Decision Tree Results
Mean Squared Error:  0.004397611358290673
R2 Score:  0.7516275066702942


Performing **Gradient Boosting Regressor**

In [ ]:
gb = GradientBoostingRegressor()
gb.fit(x_train, y_train)
y_pred_gb = gb.predict(x_test)

mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test,y_pred_gb)

print("Gradient Boosting Regressor")
print("Mean Squared Error: ", mse_gb)
print("R2 Score: ", r2_gb)

Gradient Boosting Regressor
Mean Squared Error:  0.0045510141392741165
R2 Score:  0.7429634779300696


Performing **Random forest Regressor**

In [ ]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Regressor")
print("Mean Squared Error: ",mse_rf)
print("R2 Score: ", r2_rf)

Random Forest Regressor
Mean Squared Error:  0.0038513130453944407
R2 Score:  0.7824818644161294


**Hyper Parameter** Tuning on **Random Forest Regression**

In [ ]:
rf_hyper = RandomForestRegressor(random_state=42)

param_grid = {
    'max_depth': [None, 5,10,15,20],
    'min_samples_split': [2,5,10],
    'min_samples_leaf' : [1,2,4],
    'criterion' : ['squared_error', 'absolute_error']
    }

grid_search = GridSearchCV(estimator = rf_hyper, param_grid = param_grid, cv=5, scoring = 'r2', n_jobs = -1)

grid_search.fit(x_train, y_train)
print("Best Parameters: ", grid_search.best_params_)
print("Best R2 Score on training data: ", grid_search.best_score_)
best_model = grid_search.best_estimator_
y_pred_hyper = best_model.predict(x_test)
print("Random Forest Hypertuning MSE:", mean_squared_error(y_test, y_pred_hyper))
print("Random Forest Hypertuning R2 Score:", r2_score(y_test, y_pred_hyper))

Best Parameters:  {'criterion': 'absolute_error', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best R2 Score on training data:  0.8342278900684199
Random Forest Hypertuning MSE: 0.004302501442300301
Random Forest Hypertuning R2 Score: 0.7569992153208034


From here we will be working on Rows with **Finishing Rows** in **ProductionDept column.**

**Making a Main Data Copy** for **Rows** wih **Finishing.**

In [ ]:
df_finish = df.copy()

**Selecting Rows** only with **Finishing & Quality.**

In [ ]:
df_finish = df_finish[df_finish['Work In Progress'].isna()]
df_finish.head(2)

,Record Date,Fiscal Quarter,Production Dept,Day Of Week,Team,Planned Efficiency,Standard Minute Value,Work In Progress,Over Time Minutes,Performance Bonus,Idle Minutes,Idle Workers,Style Change Count,Worker Count,Efficiency Score
2,2015-01-06,Q1,Finishing & Quality,Tuesday,Line-C2,0.8,2.9,NaN,1440,0,0.0,0,No Style Change,8.0,0.681061
3,2015-02-24,Q4,Finishing & Quality,Tuesday,Line-S2,0.7,3.9,NaN,960,0,0.0,0,No Style Change,8.0,0.325000


**Total number of Rows** with **Finishing & Quality.**

In [ ]:
df_finish.shape

(506, 15)

In [ ]:
df_finish.head(2)

,Record Date,Fiscal Quarter,Production Dept,Day Of Week,Team,Planned Efficiency,Standard Minute Value,Work In Progress,Over Time Minutes,Performance Bonus,Idle Minutes,Idle Workers,Style Change Count,Worker Count,Efficiency Score
2,2015-01-06,Q1,Finishing & Quality,Tuesday,Line-C2,0.8,2.9,NaN,1440,0,0.0,0,No Style Change,8.0,0.681061
3,2015-02-24,Q4,Finishing & Quality,Tuesday,Line-S2,0.7,3.9,NaN,960,0,0.0,0,No Style Change,8.0,0.325000


**Label Encoding** on **Certain Columns.**

In [ ]:
cols2_encode = ['Fiscal Quarter', 'Production Dept', 'Day Of Week', 'Team', 'Style Change Count']
le = LabelEncoder()

for col2 in cols2_encode:
  df_finish[col2] = le.fit_transform(df_finish[col2])

**Deleting the Column Record Date** and **Work in Progress because all values are NaN in work in Progress Column.**

In [ ]:
df_finish.drop(columns =['Record Date','Work In Progress'], inplace = True)

**Verifying all Columns** are **correct** as per **Requirement.**

In [ ]:
df_finish.head(2)

,Fiscal Quarter,Production Dept,Day Of Week,Team,Planned Efficiency,Standard Minute Value,Over Time Minutes,Performance Bonus,Idle Minutes,Idle Workers,Style Change Count,Worker Count,Efficiency Score
2,0,0,4,1,0.8,2.9,1440,0,0.0,0,0,8.0,0.681061
3,3,0,4,10,0.7,3.9,960,0,0.0,0,0,8.0,0.325000


**Counting** number of **Rows** and **Columns.**

In [ ]:
df_finish.shape

(506, 13)

**Assigning** the data for **xa**, **ya** and **Target.**  

In [ ]:
target2 = 'Efficiency Score'
xa = df_finish.drop(columns=[target2])
ya = df_finish[target2]

**Train Test Split.**

In [ ]:
xa_train, xa_test, ya_train, ya_test = train_test_split(xa,ya, test_size = 0.2, random_state =42)

**Scaling** with **Standard Scaling** and **Robust Scaling.**

In [ ]:
standard_column = ['Planned Efficiency', 'Standard Minute Value', 'Over Time Minutes', 'Idle Minutes', 'Worker Count']
robust_column = ['Performance Bonus']

scaler_standard = StandardScaler()
scaler_robust = RobustScaler()

scaler_standard.fit(xa_train[standard_column])
scaler_robust.fit(xa_train[robust_column])

xa_train[standard_column] = scaler_standard.transform(xa_train[standard_column])
xa_test[standard_column] = scaler_standard.transform(xa_test[standard_column])

xa_train[robust_column] = scaler_robust.transform(xa_train[robust_column])
xa_test[robust_column]= scaler_robust.transform(xa_test[robust_column])

In [ ]:
xa_train.tail(5)

,Fiscal Quarter,Production Dept,Day Of Week,Team,Planned Efficiency,Standard Minute Value,Over Time Minutes,Performance Bonus,Idle Minutes,Idle Workers,Style Change Count,Worker Count
275,0,0,1,0,0.712757,0.088830,-0.527531,0.0,0.0,0,0,-0.555355
657,3,0,1,1,0.712757,-1.690996,-0.162936,0.0,0.0,0,0,-0.298352
826,3,0,1,11,0.140284,0.088830,-0.262371,0.0,0.0,0,0,-0.555355
1037,1,0,0,4,0.712757,-1.690996,-0.262371,0.0,0.0,0,0,-0.555355
261,3,0,3,11,-0.432189,0.088830,5.571154,0.0,0.0,0,0,2.528678


Performing **Linear Regression.**

In [ ]:
lr2 = LinearRegression()
lr2.fit(xa_train, ya_train)
ya_pred_lr2 = lr2.predict(xa_test)

mse_lr2 = mean_squared_error(ya_test, ya_pred_lr2)
r2_lr2 = r2_score(ya_test, ya_pred_lr2)

print("Linear Regression Results")
print("Mean Squared Error: ",mse_lr2)
print("R2 Score: ", r2_lr2)

Linear Regression Results
Mean Squared Error:  0.027642072580953272
R2 Score:  0.22755313158227353


Performing **Decision Tree Regressor.**

In [ ]:
dt2 = DecisionTreeRegressor()
dt2.fit(xa_train, ya_train)

ya_pred_dt2 = dt2.predict(xa_test)

mse_dt2 = mean_squared_error(ya_test, ya_pred_dt2)
r2_dt2 = r2_score(ya_test, ya_pred_dt2)

print("Decision Tree Regressor")
print("Mean_Squared_Error: ", mse_dt2)
print("R2 Score: ", r2_dt2)

Decision Tree Regressor
Mean_Squared_Error:  0.04355761011579051
R2 Score:  -0.2172003177824786


Performing **Gradient Boosting Regressor.**

In [ ]:
gb2 = GradientBoostingRegressor()
gb2.fit(xa_train, ya_train)

ya_pred_gb2 = gb2.predict(xa_test)

mse_gb2 = mean_squared_error(ya_test, ya_pred_gb2)
r2_gb2 = r2_score(ya_test, ya_pred_gb2)

print("Gradient Boosting Regressor")
print("Mean Squared Error: ", mse_gb2)
print("R2 Score: ", r2_gb2)

Gradient Boosting Regressor
Mean Squared Error:  0.025678744844736315
R2 Score:  0.28241755454032047


Performing **Random Forest Regressor.**

In [ ]:
rf2 = RandomForestRegressor()
rf2.fit(xa_train, ya_train)

ya_pred_rf2 = rf2.predict(xa_test)

mse_rf2 = mean_squared_error(ya_test, ya_pred_rf2)
r2_rf2 = r2_score(ya_test, ya_pred_rf2)

print("Random Forest Regressor")
print("Mean Squared Error: ", mse_rf2)
print("r2 Score: ", r2_rf2)

Random Forest Regressor
Mean Squared Error:  0.023182205910913342
r2 Score:  0.352182355123446


Performing **Hyper parameter Tuning** on **Random Forest Regressor.**

In [ ]:
rf2_hyper = RandomForestRegressor(random_state=42)

param_grid2 = {
    'max_depth': [None, 5,10,15,20],
    'min_samples_split': [2,5,10],
    'min_samples_leaf' : [1,2,4],
    'criterion' : ['squared_error', 'absolute_error']
    }

grid_search2 = GridSearchCV(estimator = rf2_hyper, param_grid = param_grid2, cv=5, scoring = 'r2', n_jobs = -1)

grid_search2.fit(xa_train, ya_train)
print("Best Parameters: ", grid_search2.best_params_)
print("Best R2 Score on training data: ", grid_search2.best_score_)
best_model2 = grid_search2.best_estimator_
ya_pred_rf2_hyper = best_model2.predict(xa_test)
print("Random Forest Hypertuning Mean Sq. Error:", mean_squared_error(ya_test, ya_pred_rf2_hyper))
print("Random Forest Hypertuning R²:", r2_score(ya_test, ya_pred_rf2_hyper))

Best Parameters:  {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best R2 Score on training data:  0.1898850837255472
Random Forest Hypertuning Mean Sq. Error: 0.024034724812910593
Random Forest Hypertuning R²: 0.32835904903139645
